# Ferrofluid-Monolayer - Part III 

## Susceptibility with fluctuation formulas

In this part we want to calculate estimators for the initial susceptibility, i.e. the susceptibility at zero external magnetic field. One could carry out several simulations with different external magnetic field strenghts and get the initial susceptibility by fitting a line to the results. We want to go a more elegant way by using fluctuation formulas known from statistical mechanics.
In three dimensions the initial susceptibility $\chi_{init}$ can be calculated with zero field simulations through

\begin{equation}
    \chi_{init} = \frac{V \cdot \mu_0}{3 \cdot k_B T} \left( \langle \boldsymbol{M}^2 \rangle - \langle \boldsymbol{M} \rangle^2 \right) = \frac{\mu_0}{3 \cdot k_B T \cdot V} \left( \langle \boldsymbol{\mu}_{tot}^2 \rangle - \langle \boldsymbol{\mu}_{tot} \rangle^2 \right)
\end{equation}

where $\boldsymbol{M}$ is the magnetization vector. In direction $i$ it reads

\begin{equation}
    M_i = \frac{1}{V} \Bigg\langle \sum_{j=1}^N \mu_j^i \Bigg\rangle
\end{equation}

where $\mu_j^i$ is the $j$ th dipole moment in direction $i$.

For a quasi two dimensional System due to symmetry we have at least two different susceptibilities

\begin{equation}
    \chi_{init,||} = \frac{V \cdot \mu_0}{2 \cdot k_B T} \left( \langle \boldsymbol{M}^2_{||} \rangle - \langle \boldsymbol{M}_{||} \rangle^2 \right) = \frac{\mu_0}{2 \cdot k_B T \cdot V} \left( \langle \boldsymbol{\mu}^2_{tot,||} \rangle - \langle \boldsymbol{\mu}_{tot,||} \rangle^2 \right)
\end{equation}

and 

\begin{equation}
    \chi_{init,\perp} = \frac{V \cdot \mu_0}{k_B T} \left( \langle \boldsymbol{M}^2_{\perp} \rangle - \langle \boldsymbol{M}_{\perp} \rangle^2 \right) = \frac{\mu_0}{k_B T \cdot V} \left( \langle \boldsymbol{\mu}^2_{tot,\perp} \rangle - \langle \boldsymbol{\mu}_{tot,\perp} \rangle^2 \right)
\end{equation}

where $\boldsymbol{\mu}_{tot,||}$ and $\boldsymbol{\mu}_{tot,\perp}$ are the dipole moments of the whole system parallel and perpendicular to the monolayer plane.  
We now want to calculate and compare the initial susceptibility $\chi_{init}$ with the results of Ref. [1] for the monodisperse case. As the system itself is homogen we have the two different susceptibilities $\chi_{init,||}$ and $\chi_{init,\perp}$.

First we import all necessary packages

In [1]:
from __future__ import print_function

import espressomd

from espressomd.magnetostatics import DipolarP3M
from espressomd.magnetostatic_extensions import DLC

from espressomd.cluster_analysis import ClusterStructure
from espressomd.pair_criteria import DistanceCriterion


import numpy as np

Now we setting up all necessary simulation parameters

In [2]:
lj_sigma=1
lj_epsilon=1
lj_cut = 2**(1./6.) * lj_sigma

# magnetic field constant
mu_0 = 1.

# Particles
N = 1000

# density
rho = 0.4
# Area fraction of the mono-layer
phi = rho * np.pi * lj_sigma**2. / 4.

# Dipolar interaction parameter lambda = mu_0 m^2 /(4 pi sigma^3 kT)
dip_lambda = 1.

# Temperature
kT =1.0

# Friction coefficient
gamma=1.0

# Time step
dt =0.01

# Verlet list skin
skin =0.4

# System setup
box_size = (N * np.pi * (lj_sigma/2.)**2 /phi)**0.5

# dipole moment
dipm = np.sqrt(dip_lambda*4*np.pi*lj_sigma**3*kT / mu_0)

and the system (for more details see Part I)

In [3]:
system=espressomd.System(box_l=(box_size,box_size,box_size)) 
system.time_step=dt
system.cell_system.skin=skin
system.thermostat.set_langevin(kT=kT,gamma=gamma)
system.set_random_state_PRNG()

# Lennard Jones interaction
system.non_bonded_inter[0,0].lennard_jones.set_params(epsilon=lj_epsilon,sigma=lj_sigma,cutoff=lj_cut, shift="auto")

# Random dipole moments
dip_phi=np.random.random((N,1)) *2. * np.pi
dip_cos_theta = 2*np.random.random((N,1)) -1
dip_sin_theta = np.sin(np.arccos(dip_cos_theta))
dip = np.hstack((
   dip_sin_theta *np.sin(dip_phi),
   dip_sin_theta *np.cos(dip_phi),
   dip_cos_theta))

# Random positions in the monolayer
pos = box_size* np.hstack((np.random.random((N,2)), np.zeros((N,1))))

# Add particles
system.part.add(pos=pos,rotation=N*[(1,1,1)],dip=dip,fix=N*[(0,0,1)])

# Remove overlap between particles by means of the steepest descent method
system.integrator.set_steepest_descent(
    f_max=0,gamma=0.1,max_displacement=0.05)

while system.analysis.energy()["total"] > 5*kT*N:
    system.integrator.run(20)

# Switch to velocity Verlet integrator
system.integrator.set_vv()

# Setup dipolar P3M and dipolar layer correction
system.actors.add(DipolarP3M(accuracy=5E-4,prefactor=dip_lambda*lj_sigma**3*kT))
system.actors.add(DLC(maxPWerror=1E-4, gap_size=box_size-lj_sigma))

Dipolar P3M tune parameters: Accuracy goal = 5.00000e-04 prefactor = 1.00000e+00
System: box_l = 5.00000e+01 # charged part = 1000 Sum[q_i^2] = 1.00000e+03
Dmesh cao Dr_cut_iL   Dalpha_L     Derr         Drs_err    Dks_err    time [ms]
8    3   2.29664e-01 7.91354e+00 4.94976e-04 3.535e-04 3.464e-04 10      
8    2   2.43117e-01 7.17458e+00 4.97684e-04 3.535e-04 3.503e-04 10      
8    4   2.22938e-01 8.31450e+00 4.98869e-04 3.535e-04 3.520e-04 10      
8    5   2.21016e-01 8.43335e+00 4.91900e-04 3.535e-04 3.420e-04 11      
8    6   2.20055e-01 8.49349e+00 4.91163e-04 3.536e-04 3.409e-04 13      
16   3   1.80322e-01 1.15365e+01 4.97737e-04 3.536e-04 3.503e-04 8       
16   2   1.99162e-01 9.94221e+00 4.98121e-04 3.535e-04 3.509e-04 8       
16   4   1.72248e-01 1.23288e+01 4.96857e-04 3.536e-04 3.491e-04 8       
16   5   1.67762e-01 1.28031e+01 4.94615e-04 3.535e-04 3.459e-04 9       
16   6   1.65071e-01 1.31003e+01 4.89562e-04 3.536e-04 3.386e-04 11      
32   3   1.33128e-01 1.7

Now we equilibrate for a while

In [4]:
print("Equilibration...")
for i in range(30):
  system.integrator.run(1000)
  print("Dipolar energy: %.2f" % system.analysis.energy()["dipolar"])
  print("progress: {:6.2f} %".format((float(i)+1)/30 * 100))
print("Equilibration done")

Equilibration...
Dipolar energy: -294.13
progress:   3.33 %
Dipolar energy: -304.70
progress:   6.67 %
Dipolar energy: -301.73
progress:  10.00 %
Dipolar energy: -298.06
progress:  13.33 %
Dipolar energy: -281.86
progress:  16.67 %
Dipolar energy: -296.93
progress:  20.00 %
Dipolar energy: -295.96
progress:  23.33 %
Dipolar energy: -278.84
progress:  26.67 %
Dipolar energy: -284.42
progress:  30.00 %
Dipolar energy: -298.49
progress:  33.33 %
Dipolar energy: -273.94
progress:  36.67 %
Dipolar energy: -273.13
progress:  40.00 %
Dipolar energy: -299.91
progress:  43.33 %
Dipolar energy: -291.96
progress:  46.67 %
Dipolar energy: -285.51
progress:  50.00 %
Dipolar energy: -293.10
progress:  53.33 %
Dipolar energy: -308.94
progress:  56.67 %
Dipolar energy: -285.76
progress:  60.00 %
Dipolar energy: -281.67
progress:  63.33 %
Dipolar energy: -286.92
progress:  66.67 %
Dipolar energy: -262.49
progress:  70.00 %
Dipolar energy: -341.15
progress:  73.33 %
Dipolar energy: -319.62
progress:  76

As we need the magnetization of our system, we import <tt>MagneticDipoleMoment</tt> from <tt>observables</tt> which returns us the total dipole moment of the system which is the magnetization times the volume of the system.

In [5]:
from espressomd.observables import MagneticDipoleMoment
dipm_tot_calc = MagneticDipoleMoment(ids=system.part[:].id)

Now we set the desired number of loops for the sampling

In [6]:
# Sampling
loops = 1000

and sample the first and second moment of the magnetization or total dipole moment, by averaging over all total dipole moments occuring during the simulation

In [7]:
print('Sampling ...')
# calculate initial total dipole moment
dipm_tot_temp = dipm_tot_calc.calculate()

# initiate variables for sum of first and second moment of total dipole moment
dipm_tot_sum = dipm_tot_temp
dipm_tot_2_sum = np.square(dipm_tot_temp)

# sample dipole moment
for i in range(loops):
    system.integrator.run(100)
    dipm_tot_temp = dipm_tot_calc.calculate()
    
    # add current dipole moment
    dipm_tot_sum = np.sum((dipm_tot_sum, dipm_tot_temp), axis=0)
    dipm_tot_2_sum = np.sum((dipm_tot_2_sum, np.square(dipm_tot_temp)), axis=0)
    print("progress: {:6.2f} %".format((float(i)+1)/loops * 100))
    
# calculate average first and second moment of total dipole moment
dipm_tot = dipm_tot_sum / loops
dipm_tot_2 = dipm_tot_2_sum / loops
print("Sampling done")

Sampling ...
progress:   0.10 %
progress:   0.20 %
progress:   0.30 %
progress:   0.40 %
progress:   0.50 %
progress:   0.60 %
progress:   0.70 %
progress:   0.80 %
progress:   0.90 %
progress:   1.00 %
progress:   1.10 %
progress:   1.20 %
progress:   1.30 %
progress:   1.40 %
progress:   1.50 %
progress:   1.60 %
progress:   1.70 %
progress:   1.80 %
progress:   1.90 %
progress:   2.00 %
progress:   2.10 %
progress:   2.20 %
progress:   2.30 %
progress:   2.40 %
progress:   2.50 %
progress:   2.60 %
progress:   2.70 %
progress:   2.80 %
progress:   2.90 %
progress:   3.00 %
progress:   3.10 %
progress:   3.20 %
progress:   3.30 %
progress:   3.40 %
progress:   3.50 %
progress:   3.60 %
progress:   3.70 %
progress:   3.80 %
progress:   3.90 %
progress:   4.00 %
progress:   4.10 %
progress:   4.20 %
progress:   4.30 %
progress:   4.40 %
progress:   4.50 %
progress:   4.60 %
progress:   4.70 %
progress:   4.80 %
progress:   4.90 %
progress:   5.00 %
progress:   5.10 %
progress:   5.20 %

progress:  43.20 %
progress:  43.30 %
progress:  43.40 %
progress:  43.50 %
progress:  43.60 %
progress:  43.70 %
progress:  43.80 %
progress:  43.90 %
progress:  44.00 %
progress:  44.10 %
progress:  44.20 %
progress:  44.30 %
progress:  44.40 %
progress:  44.50 %
progress:  44.60 %
progress:  44.70 %
progress:  44.80 %
progress:  44.90 %
progress:  45.00 %
progress:  45.10 %
progress:  45.20 %
progress:  45.30 %
progress:  45.40 %
progress:  45.50 %
progress:  45.60 %
progress:  45.70 %
progress:  45.80 %
progress:  45.90 %
progress:  46.00 %
progress:  46.10 %
progress:  46.20 %
progress:  46.30 %
progress:  46.40 %
progress:  46.50 %
progress:  46.60 %
progress:  46.70 %
progress:  46.80 %
progress:  46.90 %
progress:  47.00 %
progress:  47.10 %
progress:  47.20 %
progress:  47.30 %
progress:  47.40 %
progress:  47.50 %
progress:  47.60 %
progress:  47.70 %
progress:  47.80 %
progress:  47.90 %
progress:  48.00 %
progress:  48.10 %
progress:  48.20 %
progress:  48.30 %
progress:  4

progress:  86.40 %
progress:  86.50 %
progress:  86.60 %
progress:  86.70 %
progress:  86.80 %
progress:  86.90 %
progress:  87.00 %
progress:  87.10 %
progress:  87.20 %
progress:  87.30 %
progress:  87.40 %
progress:  87.50 %
progress:  87.60 %
progress:  87.70 %
progress:  87.80 %
progress:  87.90 %
progress:  88.00 %
progress:  88.10 %
progress:  88.20 %
progress:  88.30 %
progress:  88.40 %
progress:  88.50 %
progress:  88.60 %
progress:  88.70 %
progress:  88.80 %
progress:  88.90 %
progress:  89.00 %
progress:  89.10 %
progress:  89.20 %
progress:  89.30 %
progress:  89.40 %
progress:  89.50 %
progress:  89.60 %
progress:  89.70 %
progress:  89.80 %
progress:  89.90 %
progress:  90.00 %
progress:  90.10 %
progress:  90.20 %
progress:  90.30 %
progress:  90.40 %
progress:  90.50 %
progress:  90.60 %
progress:  90.70 %
progress:  90.80 %
progress:  90.90 %
progress:  91.00 %
progress:  91.10 %
progress:  91.20 %
progress:  91.30 %
progress:  91.40 %
progress:  91.50 %
progress:  9

Now we can calculate estimators of the susceptibility parallel and perpendicular to the monolayer plane

In [8]:
# susceptibility parallel to monolayer plane
chi_p =  mu_0 /(box_size**2. * 2.) * ( (dipm_tot_2[0] * dipm**2. - (dipm_tot[0] * dipm)**2.) + (dipm_tot_2[1] * dipm**2. - (dipm_tot[1] * dipm)**2.) )

In [9]:
# susceptibility perpendicular to monolayer plane
chi_perp = mu_0 /(box_size**2.) * (dipm_tot_2[2] * dipm**2. - (dipm_tot[2] * dipm)**2.)

In [11]:
print(mu_0)

1


and print the results

In [10]:
print('chi_parallel\t\t= {}'.format(chi_p))
print('chi_perpendicular\t= {}'.format(chi_perp))

chi_parallel		= 2.88606868322
chi_perpendicular	= 0.842376069982


Copared with table 1 in Ref. [1] they should be very similar

[1] Tamás Kristóf and István Szalai. “Magnetic properties in monolayers of a model
polydisperse ferrofluid”. In: Phys. Rev. E 72 (4 Oct. 2005), p. 041105. doi: 10.1103/
PhysRevE.72.041105. url: https://link.aps.org/doi/10.1103/PhysRevE.72.
041105.